The model trained in this notebook is used in exercise 2 of the embeddings course (matrix factorization). It's identical to the matrix factorization model we train in tutorial 2, except that it adds some L2 regularization to our movie and user embeddings.

In [ ]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import tensorflow as tf
from tensorflow import keras
import os
import random

RUNNING_ON_KERNELS = 'KAGGLE_WORKING_DIR' in os.environ
input_dir = '../input' if RUNNING_ON_KERNELS else '../input/movielens_preprocessed'
ratings_path = os.path.join(input_dir, 'rating.csv')
df = pd.read_csv(ratings_path, usecols=['userId', 'movieId', 'rating', 'y'])

tf.set_random_seed(1); np.random.seed(1); random.seed(1)

In [ ]:
movie_embedding_size = user_embedding_size = 8
user_id_input = keras.Input(shape=(1,), name='user_id')
movie_id_input = keras.Input(shape=(1,), name='movie_id')

movie_r12n = keras.regularizers.l1_l2(l1=0, l2=1e-6)
user_r12n = keras.regularizers.l1_l2(l1=0, l2=1e-7)
user_embedded = keras.layers.Embedding(df.userId.max()+1, user_embedding_size,
                                       embeddings_initializer='glorot_uniform',
                                       embeddings_regularizer=user_r12n,
                                       input_length=1, name='user_embedding')(user_id_input)
movie_embedded = keras.layers.Embedding(df.movieId.max()+1, movie_embedding_size, 
                                        embeddings_initializer='glorot_uniform',
                                        embeddings_regularizer=movie_r12n,
                                        input_length=1, name='movie_embedding')(movie_id_input)

dotted = keras.layers.Dot(2)([user_embedded, movie_embedded])
out = keras.layers.Flatten()(dotted)

l2_model = keras.Model(
    inputs = [user_id_input, movie_id_input],
    outputs = out,
)
l2_model.compile(
    tf.train.AdamOptimizer(0.005),
    loss='MSE',
    metrics=['MAE', 'MSE'],
)
l2_model.summary(line_length=88)

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
user_id (InputLayer)            (None, 1)            0                                            
__________________________________________________________________________________________________
movie_id (InputLayer)           (None, 1)            0                                            
__________________________________________________________________________________________________
user_embedding (Embedding)      (None, 1, 8)         1107952     user_id[0][0]                    
__________________________________________________________________________________________________
movie_embedding (Embedding)     (None, 1, 8)         213952      movie_id[0][0]                   
__________________________________________________________________________________________________
dot (Dot) 

In [ ]:
l2_model.fit(
    [df.userId, df.movieId],
    df.y,
    batch_size=10**4,
    epochs=10,
    verbose=2,
    validation_split=.05,
);

Train on 19000249 samples, validate on 1000014 samples
Epoch 1/10
 - 94s - loss: 0.7794 - mean_absolute_error: 0.6712 - val_loss: 0.6997 - val_mean_absolute_error: 0.6289
Epoch 2/10
 - 88s - loss: 0.6695 - mean_absolute_error: 0.6132 - val_loss: 0.6722 - val_mean_absolute_error: 0.6129
Epoch 3/10
 - 93s - loss: 0.6428 - mean_absolute_error: 0.5984 - val_loss: 0.6655 - val_mean_absolute_error: 0.6081
Epoch 4/10
 - 87s - loss: 0.6336 - mean_absolute_error: 0.5928 - val_loss: 0.6644 - val_mean_absolute_error: 0.6064
Epoch 5/10
 - 82s - loss: 0.6302 - mean_absolute_error: 0.5904 - val_loss: 0.6644 - val_mean_absolute_error: 0.6055
Epoch 6/10
 - 89s - loss: 0.6287 - mean_absolute_error: 0.5892 - val_loss: 0.6649 - val_mean_absolute_error: 0.6053
Epoch 7/10
 - 91s - loss: 0.6280 - mean_absolute_error: 0.5884 - val_loss: 0.6651 - val_mean_absolute_error: 0.6048
Epoch 8/10
 - 87s - loss: 0.6276 - mean_absolute_error: 0.5879 - val_loss: 0.6655 - val_mean_absolute_error: 0.6049
Epoch 9/10
 - 84s

Wow, this is impressively better than the same model without r12n. Well, actually, the final errors are pretty close (.6050 vs. .6047) but the r12n model at least converges a lot faster? Thoooough, maybe the emb init thing is a confound? 
... Yup, huge confound. glorot_uniform greatly improves convergence rate of unregularized tut-2 model, putting it on par with this model. Oh well.

Should experiment with l2 strength. I think the values here (settled on during some earlier experiment) maybe leaned on the side of going a little overboard, for the sake of more conservative predictions (minimizing how foolish we look, aggressively targetting the obscure movie problem) rather than optimizing for validation loss. 

Still todo:

- Visually compare error over time (load serialized training history from 2-factorization tutorial, or just train a vanilla model here from scratch)

In [ ]:
l2_model.save('movie_svd_model_8_r12n.h5')